# Run Delta segmentation & tracking pipeline

Here we will use the Delta2 package to segment and track timelapse data of microcolonies using a deep learning based workflow.  
You can find extensive documentation on Delta [here](https://delta.readthedocs.io).

In addition we will need the `json` package to edit the configuration files.

In [ ]:
import pathlib
import delta
import json

Deep Learning networks can run on a CPU, but are much much faster on a GPU.  
Let's see if we can use a GPU (you should see get a line with `device_type='GPU'):

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices()

---

## Setup Folders
Set the path to the one you used in `0_download_model_delta`.  
We will also create a `ProcessedData` folder where we will store the output

In [ ]:
root = pathlib.Path(pathlib.Path.home(), 'workdir/')
proj_dir = (root / 'Project2B')
model_dir = proj_dir / 'models' #location of model
data_dir = proj_dir / 'RawData' #location of raw data
output_dir = proj_dir / 'ProcessedData' #location of output data
(output_dir).mkdir(exist_ok=True) #create output data folder

!ls $proj_dir

---

## Modify Config Files

The Delta Pipeline is controlled using `.json` [config files](https://delta.readthedocs.io/en/latest/usage/config_desc.html). By default two config files are provide: one for 1D mother machine data: `config_mothermachine.json` and one for 2D (microcolony or flowcell) data: `config_2D.json`.

You need to modify these config files to point Delta to the correct data folder.  
We also need to specify the correct path to the pre-trained models.

Here we will open the config file using the [`json`](https://docs.python.org/3/library/json.html) package:

In [ ]:
# select config file to modify ('2D' or 'mothermachine'):
config_filename = proj_dir / 'config_2D.json'
with open(config_filename) as f:
    config = json.load(f)

Now we have to modify the relevant fields to point to the correct paths.

*Technical note: reminder: `delta` needs paths specified as strings, we can make a quick function to do this:*

In [ ]:
def to_str(posixpath):
    return str(posixpath.resolve())    

In [ ]:
#point to model location
config['model_file_seg'] = to_str(model_dir / 'unet_pads_seg.hdf5')    
config['model_file_track'] = to_str(model_dir / 'unet_pads_track.hdf5')    

#only needed for model training, in that case, point to location of training data
config['training_set_seg'] = ''
config['training_set_track'] = ''

#point to raw data location
config['eval_movie'] = to_str(data_dir)

#specify output formats
config['save_format'] = ["pickle", "movie"]    

Now we can write the config file back to disk:

In [ ]:
new_config_filename = proj_dir / 'config_2D_local.json'
with open(new_config_filename, 'w') as f:
    json.dump(config, f, indent=2)

---

## Setup the pipeline

Now we setup the pipeline, most importantly, you have to specify the naming format using the `xpreader` function.  
See [here](https://delta.readthedocs.io/en/latest/usage/pipeline_desc.html) for detailed instructions. 


In [ ]:
# Load config
delta.config.load_config(new_config_filename)

# set path to raw data to analyze:
file_path = to_str(data_dir)

# Init reader and specify file naming scheme
xpreader = delta.utils.xpreader(
            file_path,
            prototype = 'Pos%03i_Frm%03i_Ch%02i.tif',
            fileorder = 'ptc',
            filenamesindexing=1
            )

# Print experiment parameters to make sure it initialized properly:
print("""Initialized experiment reader:
    - %d positions
    - %d imaging channels
    - %d timepoints"""%(xpreader.positions, xpreader.channels, xpreader.timepoints)
)

# Init pipeline:
xp = delta.pipeline.Pipeline(xpreader, resfolder=output_dir)   

---

## Run Pipeline on GPU
Running the full analysis without a GPU takes a long time (150min on 2017 MacBook Pro with 2.8 GHz Quad-Core i7; using a RTX8000 GPU the same dataset is however processed in 3min!).

If you have a GPU, you can run the code below. If you only have a CPU, you can save some time by going to the next section where we will only run first 4 frames and then download the pre-processed data.



In [ ]:
# Run Pipeline
xp.process()

Let's check the output: we should have a `.mp4` movie for visual inspection, and a `.pkl` file containing all the data. 

In [ ]:
!ls $output_dir

---

## Next Step: Post-process Delta output
Continue with the next notebook `2_post_processing_delta`

---

## Extra material: Batch processing 

Above we processed only a single position. If you want to process multiple positions you have two options:

1. Put all data in the same folder and indicate position in filename. This works well, but only works if all positions have exact same size. 
2. Loop over all positions. For this you can look at the provided notebook `extra_batch_script`

---

## Extra material: CPU fallback

**DO NOT RUN THIS SECTION if you already processed the full-pipeline above**

If you have no GPU, you can use this section to just analyze the first 4 frames, so you can see the process. We will then load pre-processed data in the next notebook.

Uncomment the code below to analyze the first 4 frames:

In [ ]:
#xp.process(frames=list(range(4)))

Now we download pre-processed data if needed. Uncomment the lines below to download the data


In [ ]:
# file_name = 'results.zip'
# file_path = output_dir / file_name

# !wget -q  -O $file_path https://drive.switch.ch/index.php/s/8LBYv90PoThxh3r/download
# !(cd $output_dir && unzip -o -q -j $file_name)

Let's check the output: we should have a `.mp4` movie for visual inspection, and a `.pkl` file containing all the data. 

In [ ]:
!ls $output_dir

Finally we can remove the zip file again:

In [ ]:
!rm $file_path